In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 40]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
3022  prop-1       40   25    2    0    8   59   150   0   8  ...  0.416667   
3023  prop-1       40    7    3    0    2   14     9   0   2  ...  1.000000   
3024  prop-1       40    2    1    0    4    2     1   0   4  ...  0.000000   
3025  prop-1       40    1    1    0    0    5     0   0   0  ...  0.000000   
3026  prop-1       40    3    1    0    1    4     1   1   0  ...  1.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
7070  prop-1       40    4    3    1    5   17     2   1   4  ...  0.800000   
7071  prop-1       40    4    1    0    9   20     2   1   8  ...  0.750000   
7072  prop-1       40    2    4    0    7    8     1   1   6  ...  0.000000   
7073  prop-1       40    6    3    0    7   21     5   0   7  ...  1.000000   
7074  prop-1       40    3    1    0    4   11     3   1   3  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
3022    0  0.620690  0.168889   2   14  17.400000      14  1.9600    0  
3023    0  0.933333  0.285714   1    2   5.000000       1  0.5714    0  
3024    0  0.000000  0.666667   0    0   0.000000       1  1.0000    0  
3025    0  0.000000  0.000000   0    0  12.000000       0  0.0000    1  
3026    0  0.000000  0.666667   0    0   3.333333       1  0.6667    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
7070    0  0.866667  0.833333   1    1  23.250000       4  1.5000    0  
7071    1  0.000000  0.444444   0    0  18.000000       3  1.0000    0  
7072    0  0.937500  0.750000   1    1  14.500000       2  1.0000    0  
7073    0  0.886364  0.366667   2    6  20.666667       4  2.0000    0  
7074    0  0.000000  0.333333   0    0  22.333333       3  1.6667    0  

[4053 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
6570    5    5    0    4    9    10   2   2    5  1.250000   28  1.000000   
4891    1    2    1    6    2     0   5   1    1  2.000000    5  0.000000   
5101    2    3    0    9   10     1   0   9    2  1.000000   76  0.000000   
3965   20    3    0   47   35    78  32  17   20  0.729323  316  1.000000   
4681    7    1    0    2   18     3   1   1    6  0.833333  126  0.600000   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
6164    3    2    0    5    7     3   1   4    1  1.000000   47  1.000000   
3096    3    1    0    0   10     1   0   0    1  0.500000  326  1.000000   
3340   19    3    0    4   38   107   0   4   16  0.833333  254  0.500000   
6881    2    1    0    5    4     1   0   5    2  2.000000   19  0.000000   
3909   10    1    0   10   19    41   5   5   10  0.925926   73  0.666667   

      moa       mfa       cam  ic  cbm         amc  max_cc  avg_cc  
6570    0  0.962963  0.650000   1    1    4.400000       1  0.2000  
4891    0  1.000000  1.000000   0    0    4.000000       0  0.0000  
5101    0  0.947368  0.666667   0    0   36.000000       5  2.5000  
3965    5  0.644444  0.212500   0    0   14.450000       5  1.4500  
4681    0  0.000000  0.285714   1    1   16.285714       3  1.7143  
...   ...       ...       ...  ..  ...         ...     ...     ...  
6164    1  0.800000  0.666667   1    1   14.333333       2  1.0000  
3096    0  0.000000  0.500000   0    0  107.333333       2  0.6667  
3340    0  0.768116  0.175439   3   17   12.052632       8  1.4737  
6881    0  0.000000  0.666667   0    0    8.500000       2  1.0000  
3909    0  0.000000  0.280000   0    0    6.000000       1  0.9000  

[3242 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc   dam  moa  \
4443    2    1    0    4    2     1   4   0    2  2.000000    2  0.00    0   
4251    6    3    1   21   34     9   1  20    6  0.850000  252  0.75    3   
4995   13    1    0   17   28    70   3  15   11  0.950000  127  0.40    0   
6796    4    1    0   12   25     2   0  12    2  0.750000  173  0.50    0   
4924    6    1    0    2    9     1   2   1    0  0.872727  143  0.00    3   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...   ...  ...   
5247    3    3    0   22   41     1   0  22    2  1.000000  166  0.25    0   
3421    2    3    0    8   18     1   0   8    1  2.000000   67  1.00    0   
4372    6    1    0   35   42     3  18  17    4  0.700000  314  0.50    0   
4206   73    1    0   35   73  2628  30   6   73  2.000000   73  0.00    0   
5330    2    1    0   11   10     1   0  11    2  2.000000   36  0.00    0   

           mfa       cam  ic  cbm        amc  max_cc  avg_cc  
4443  0.000000  1.000000   0    0   0.000000       1  1.0000  
4251  0.916667  0.270833   2   16  40.333333       1  0.8333  
4995  0.000000  0.333333   0    0   8.384615       2  1.0769  
6796  0.000000  0.500000   0    0  41.250000      10  3.2500  
4924  0.000000  0.360000   0    0  21.000000       8  3.0000  
...        ...       ...  ..  ...        ...     ...     ...  
5247  0.857143  0.666667   2    2  53.000000       9  3.0000  
3421  0.928571  0.750000   1    1  32.000000       7  3.5000  
4372  0.000000  0.440000   0    0  51.000000       3  1.6667  
4206  0.000000  0.147260   0    0   0.000000       1  1.0000  
5330  0.000000  0.750000   0    0  17.000000       2  1.0000  

[811 rows x 20 columns]

In [13]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [14]:
y_train

bug
6570    0
4891    0
5101    0
3965    0
4681    0
...   ...
6164    0
3096    0
3340    0
6881    0
3909    0

[3242 rows x 1 columns]

In [15]:
y_test

bug
4443    0
4251    0
4995    0
6796    0
4924    0
...   ...
5247    0
3421    0
4372    0
4206    0
5330    0

[811 rows x 1 columns]

In [16]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [17]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [18]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [19]:
y = y_test['bug'].tolist()
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [20]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv40/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4687 - mse: 0.4687 - mae: 0.1786 - root_mean_squared_error: 0.6846 - mean_squared_logarithmic_error: 0.0989
Epoch 2/100
33/33 [==============================] - 0s 898us/step - loss: 0.4686 - mse: 0.4686 - mae: 0.1778 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0988
Epoch 3/100
33/33 [==============================] - 0s 901us/step - loss: 0.4686 - mse: 0.4686 - mae: 0.1776 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 4/100
33/33 [==============================] - 0s 967us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1775 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 5/100
33/33 [==============================] - 0s 996us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 6/100
33/33 [==============================] - 0s 976us/step 

33/33 [==============================] - 0s 855us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 88/100
33/33 [==============================] - 0s 915us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 89/100
33/33 [==============================] - 0s 952us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 90/100
33/33 [==============================] - 0s 891us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 91/100
33/33 [==============================] - 0s 874us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 92/100
33/33 [==============================] - 0s 925us/step - loss: 0.4685 

33/33 [==============================] - 0s 848us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 73/100
33/33 [==============================] - 0s 772us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 74/100
33/33 [==============================] - 0s 854us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 75/100
33/33 [==============================] - 0s 819us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 76/100
33/33 [==============================] - 0s 827us/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 77/100
33/33 [==============================] - 0s 752us/step - loss: 0.4685 

33/33 [==============================] - 0s 815us/step - loss: 1.0909 - mse: 1.0909 - mae: 0.1912 - root_mean_squared_error: 1.0445 - mean_squared_logarithmic_error: 0.1032
Epoch 57/100
33/33 [==============================] - 0s 771us/step - loss: 1.0228 - mse: 1.0228 - mae: 0.1904 - root_mean_squared_error: 1.0113 - mean_squared_logarithmic_error: 0.1031
Epoch 58/100
33/33 [==============================] - 0s 770us/step - loss: 0.9673 - mse: 0.9673 - mae: 0.1898 - root_mean_squared_error: 0.9835 - mean_squared_logarithmic_error: 0.1030
Epoch 59/100
33/33 [==============================] - 0s 722us/step - loss: 0.9193 - mse: 0.9193 - mae: 0.1892 - root_mean_squared_error: 0.9588 - mean_squared_logarithmic_error: 0.1029
Epoch 60/100
33/33 [==============================] - 0s 773us/step - loss: 0.8678 - mse: 0.8678 - mae: 0.1885 - root_mean_squared_error: 0.9316 - mean_squared_logarithmic_error: 0.1027
Epoch 61/100
33/33 [==============================] - 0s 738us/step - loss: 0.8245 

33/33 [==============================] - 0s 844us/step - loss: 0.4689 - mse: 0.4689 - mae: 0.1787 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0990
Epoch 41/100
33/33 [==============================] - 0s 800us/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1786 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0989
Epoch 42/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1786 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0989
Epoch 43/100
33/33 [==============================] - 0s 776us/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1785 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0989
Epoch 44/100
33/33 [==============================] - 0s 781us/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1785 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0989
Epoch 45/100
33/33 [==============================] - 0s 788us/step - loss: 0.4688 - 

...

33/33 [==============================] - 0s 1ms/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 89/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 90/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 91/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 92/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4685 - mse: 0.4685 - mae: 0.1774 - root_mean_squared_error: 0.6845 - mean_squared_logarithmic_error: 0.0987
Epoch 93/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4685 - mse: 0.468

33/33 [==============================] - 0s 863us/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1777 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0989
Epoch 31/100
33/33 [==============================] - 0s 898us/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1776 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0989
Epoch 32/100
33/33 [==============================] - 0s 838us/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1776 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0988
Epoch 33/100
33/33 [==============================] - 0s 838us/step - loss: 0.4688 - mse: 0.4688 - mae: 0.1776 - root_mean_squared_error: 0.6847 - mean_squared_logarithmic_error: 0.0988
Epoch 34/100
33/33 [==============================] - 0s 816us/step - loss: 0.4687 - mse: 0.4687 - mae: 0.1776 - root_mean_squared_error: 0.6846 - mean_squared_logarithmic_error: 0.0988
Epoch 35/100
33/33 [==============================] - 0s 823us/step - loss: 0.4687 

33/33 [==============================] - 0s 873us/step - loss: 1398.2650 - mse: 1398.2650 - mae: 13.5215 - root_mean_squared_error: 37.3934 - mean_squared_logarithmic_error: 3.9817
Epoch 16/100
33/33 [==============================] - 0s 976us/step - loss: 1201.7692 - mse: 1201.7692 - mae: 12.1970 - root_mean_squared_error: 34.6665 - mean_squared_logarithmic_error: 3.6430
Epoch 17/100
33/33 [==============================] - 0s 859us/step - loss: 1037.5044 - mse: 1037.5044 - mae: 10.9327 - root_mean_squared_error: 32.2103 - mean_squared_logarithmic_error: 3.3074
Epoch 18/100
33/33 [==============================] - 0s 858us/step - loss: 892.3575 - mse: 892.3575 - mae: 9.7709 - root_mean_squared_error: 29.8724 - mean_squared_logarithmic_error: 2.9909
Epoch 19/100
33/33 [==============================] - 0s 864us/step - loss: 765.7943 - mse: 765.7943 - mae: 8.7904 - root_mean_squared_error: 27.6730 - mean_squared_logarithmic_error: 2.7230
Epoch 20/100
33/33 [=============================

Epoch 1/100
33/33 [==============================] - 1s 3ms/step - loss: 1.9174 - mse: 1.9174 - mae: 0.2146 - root_mean_squared_error: 1.3847 - mean_squared_logarithmic_error: 0.1085
Epoch 2/100
33/33 [==============================] - 0s 1ms/step - loss: 0.6842 - mse: 0.6842 - mae: 0.1885 - root_mean_squared_error: 0.8272 - mean_squared_logarithmic_error: 0.1014
Epoch 3/100
33/33 [==============================] - 0s 1ms/step - loss: 0.5066 - mse: 0.5066 - mae: 0.1813 - root_mean_squared_error: 0.7117 - mean_squared_logarithmic_error: 0.1005
Epoch 4/100
33/33 [==============================] - 0s 958us/step - loss: 0.4848 - mse: 0.4848 - mae: 0.1800 - root_mean_squared_error: 0.6963 - mean_squared_logarithmic_error: 0.1000
Epoch 5/100
33/33 [==============================] - 0s 987us/step - loss: 0.4746 - mse: 0.4746 - mae: 0.1792 - root_mean_squared_error: 0.6889 - mean_squared_logarithmic_error: 0.0995
Epoch 6/100
33/33 [==============================] - 0s 1ms/step - loss: 0.4702 -

33/33 [==============================] - 0s 886us/step - loss: 0.4675 - mse: 0.4675 - mae: 0.1775 - root_mean_squared_error: 0.6837 - mean_squared_logarithmic_error: 0.0985
Epoch 90/100
33/33 [==============================] - 0s 891us/step - loss: 0.4675 - mse: 0.4675 - mae: 0.1775 - root_mean_squared_error: 0.6837 - mean_squared_logarithmic_error: 0.0985
Epoch 91/100
33/33 [==============================] - 0s 908us/step - loss: 0.4675 - mse: 0.4675 - mae: 0.1775 - root_mean_squared_error: 0.6837 - mean_squared_logarithmic_error: 0.0985
Epoch 92/100
33/33 [==============================] - 0s 856us/step - loss: 0.4675 - mse: 0.4675 - mae: 0.1775 - root_mean_squared_error: 0.6837 - mean_squared_logarithmic_error: 0.0985
Epoch 93/100
33/33 [==============================] - 0s 909us/step - loss: 0.4675 - mse: 0.4675 - mae: 0.1775 - root_mean_squared_error: 0.6837 - mean_squared_logarithmic_error: 0.0985
Epoch 94/100
33/33 [==============================] - 0s 904us/step - loss: 0.4675 

In [21]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv40/'

In [22]:
total_training_time

[4.066060781478882,
 3.8635733127593994,
 3.543571710586548,
 3.539177894592285,
 3.63570237159729,
 3.5511693954467773,
 3.518524408340454,
 3.5194318294525146,
 3.7838399410247803,
 4.318371772766113,
 4.369917869567871,
 4.3244640827178955,
 4.314377069473267,
 4.271876573562622,
 4.172784090042114,
 4.3896026611328125,
 4.305194854736328,
 4.3213982582092285,
 4.291289806365967,
 4.348595380783081,
 4.474075555801392,
 4.61103081703186,
 4.4999449253082275,
 4.782260417938232,
 4.4874427318573,
 4.767763614654541,
 4.492021799087524,
 4.428846597671509,
 4.474684715270996,
 4.580819845199585]

In [23]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 4.20159383614858


In [24]:
total_training_loss_history

[{'loss': [0.46873724460601807,
   0.46857762336730957,
   0.4685520827770233,
   0.4685443341732025,
   0.4685403108596802,
   0.468538761138916,
   0.46853819489479065,
   0.46853798627853394,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792667388916,
   0.46853792

In [25]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 1.2601243952910106
avg mae of training: 0.19273237586021424


In [26]:
total_scores

[[0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6536145210266113,
  0.6536145210266113,
  0.2076079249382019,
  0.8084642887115479,
  0.11495700478553772],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.5651899576187134,
  0.5651899576187134,
  0.29237499833106995,
  0.7517911791801453,
  0.10051626712083817],
 [0.6638094782829285,
  0.6638094782829285,
  0.21071460843086243,
  0.8147450685501099,
  0.11715351790189743],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.114878594

In [4]:
a = [[0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6536145210266113,
  0.6536145210266113,
  0.2076079249382019,
  0.8084642887115479,
  0.11495700478553772],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.5651899576187134,
  0.5651899576187134,
  0.29237499833106995,
  0.7517911791801453,
  0.10051626712083817],
 [0.6638094782829285,
  0.6638094782829285,
  0.21071460843086243,
  0.8147450685501099,
  0.11715351790189743],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6541245579719543,
  0.6541245579719543,
  0.2086179256439209,
  0.8087796568870544,
  0.11525560170412064],
 [0.9291663765907288,
  0.9291663765907288,
  0.22558781504631042,
  0.9639327526092529,
  0.12433668226003647],
 [0.5870303511619568,
  0.5870303511619568,
  0.3318365812301636,
  0.766179084777832,
  0.10447815805673599],
 [4.187899589538574,
  4.187899589538574,
  0.3026689887046814,
  2.046435832977295,
  0.1511337012052536],
 [0.6538639664649963,
  0.6538639664649963,
  0.20780843496322632,
  0.8086185455322266,
  0.11510340869426727],
 [10.031414031982422,
  10.031414985656738,
  0.34607836604118347,
  3.167240858078003,
  0.153558149933815],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.7040570974349976,
  0.7040570974349976,
  0.2192743718624115,
  0.8390811085700989,
  0.11277348548173904],
 [0.7981453537940979,
  0.7981453537940979,
  0.23965482413768768,
  0.8933898210525513,
  0.13027580082416534],
 [0.6522484421730042,
  0.6522484421730042,
  0.2071915864944458,
  0.8076189756393433,
  0.1141744926571846],
 [2.9086084365844727,
  2.9086084365844727,
  0.27175047993659973,
  1.705464243888855,
  0.13860361278057098],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [1.4329512119293213,
  1.4329512119293213,
  0.24621756374835968,
  1.1970593929290771,
  0.1309502273797989],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [23.234697341918945,
  23.234697341918945,
  0.5108351111412048,
  4.82023811340332,
  0.18950232863426208],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [0.6535142064094543,
  0.6535142064094543,
  0.20715166628360748,
  0.8084022402763367,
  0.1148785948753357],
 [1.174178123474121,
  1.174178123474121,
  0.2317740023136139,
  1.0835950374603271,
  0.12597835063934326],
 [0.6521952152252197,
  0.6521952152252197,
  0.2063637375831604,
  0.8075860142707825,
  0.1141265258193016]]

In [5]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10

0.21622254550457

In [3]:
test_mae

[0.20715166628360748,
 0.20715166628360748,
 0.20715166628360748,
 0.2076079249382019,
 0.20715166628360748,
 0.20715166628360748,
 0.29237499833106995,
 0.21071460843086243,
 0.20715166628360748,
 0.2086179256439209,
 0.22558781504631042,
 0.3318365812301636,
 0.3026689887046814,
 0.20780843496322632,
 0.34607836604118347,
 0.20715166628360748,
 0.2192743718624115,
 0.23965482413768768,
 0.2071915864944458,
 0.27175047993659973,
 0.20715166628360748,
 0.24621756374835968,
 0.20715166628360748,
 0.5108351111412048,
 0.20715166628360748,
 0.20715166628360748,
 0.20715166628360748,
 0.20715166628360748,
 0.2317740023136139,
 0.2063637375831604]

In [30]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 1.9659626245498658
avg mae: 0.23864429940780005
avg are: 0.12600950251022974


In [31]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.46853792667388916
median testing mse: 0.6535142064094543


In [32]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.17753049731254578
median testing mae: 0.20739975571632385


In [33]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.1257706582546234


In [34]:
test_ARE

[0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.1216057,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.122920945,
 0.12577066,
 0.12577066,
 0.12618168,
 0.12577066,
 0.12433209,
 0.12823674,
 0.12577066,
 0.1294698,
 0.12577066,
 0.13193588,
 0.12577066,
 0.12864776,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066]

In [6]:
sum([0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.1216057,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.122920945,
 0.12577066,
 0.12577066,
 0.12618168,
 0.12577066,
 0.12433209,
 0.12823674,
 0.12577066,
 0.1294698,
 0.12577066,
 0.13193588,
 0.12577066,
 0.12864776,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066,
 0.12577066][:10])/10

0.125354164